In [1]:
# Cell 1: Load libraries

#Imort modules
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.image as img
from skimage.transform import resize
import pandas as pd
import s2sphere

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load the Tensorflow Hub modul
module = hub.Module("https://www.kaggle.com/models/google/planet-v2/frameworks/TensorFlow1/variations/planet-vision-classifier-planet-v2/versions/1")
height, width = hub.get_expected_image_size(module)

# load labelmap
labelmap = pd.read_csv("planet_v2_labelmap.csv")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [2]:
# Cell 2: Needed Functions

# find x maxima
def find_x_maxima(array, x):
    indices = np.argsort(array)[-x:][::-1]  # indices of maximas
    maxima = array[indices]  # values of maximas
    return indices, maxima

# Crates txt for Geoplots
def text_datei(idx_nummer,score,dateiname):
    with open("results/Geoplot-"+dateiname+".txt",'w') as datei:
        for i in range(5):
            id_nummer=idx_nummer[i]

            # get cellnumber in Labelmap
            corresponding_row = labelmap[labelmap['id'] == id_nummer]
            s2_cell_id_hex = corresponding_row['S2CellId'].values[0]

            # Get Coordinates of the Cells
            s2_cell_id = s2sphere.CellId.from_token(s2_cell_id_hex)
            s2_cell = s2sphere.Cell(s2_cell_id)
            vertices = [s2sphere.LatLng.from_point(s2_cell.get_vertex(i)) for i in range(4)]
            latitudes, longitudes = zip(*[(vertex.lat().degrees, vertex.lng().degrees) for vertex in vertices])
            latitudes_array = np.array(latitudes)
            longitudes_array = np.array(longitudes)

            # Write txt
            datei.write(str(latitudes_array[:]) + '\n')
            datei.write(str(longitudes_array[:]) + '\n')
            datei.write(str(score[i]) + '\n')
            datei.write(str(id_nummer)+ '\n')
    
# Function for Geoplots
def bildanalyse_geoplots(imagenamen):
    
    # Load and resize images
    images = np.zeros([len(imagenamen),299,299,3])
    for i in range(len(imagenamen)):
        image = img.imread("images/"+ imagenamen[i])
        image = image[:,:,0:3]
        image_res = resize(image, (299, 299))
        images[i,:,:,:] = image_res[:,:,:]
    print("Images loaded")

    # Run PlaNet
    features = module(images)
    print('Network done')

    # create Numpy-Array
    with tf.compat.v1.Session() as sess:
        features_np = sess.run(features)
    print('NP-Array created')

    # Generate txt-data
    for i in range(len(imagenamen)):
        maxima_id,maxima_score = find_x_maxima(features_np[i,:],5)
        text_datei(maxima_id,maxima_score,imagenamen[i])
    print('txt generated')


# Function for Superpixel
def bildanalyse_superpixel(imagename,nmax,endung):
    
    # Load images
    images = np.zeros([nmax+1,299,299,3])
    # original image
    image = img.imread("images/Segmented_imgs_"+imagename+"/Segmented"+endung)
    image = image[:,:,0:3]
    image_res = resize(image, (299, 299))
    images[0,:,:,:] = image_res[:,:,:]
    # superpixel images
    for i in range(nmax):
        image = img.imread("images/Segmented_imgs_"+imagename+"/Segmented_"+str(i+1)+".png")
        image = image[:,:,0:3]
        image_res = resize(image, (299, 299))
        images[i+1,:,:,:] = image_res[:,:,:]
    print('- Images loaded')
    
    # Run PlaNet
    features = module(images)
    print('- Network done')

    # create Numpy-Array
    with tf.compat.v1.Session() as sess:
        features_np = sess.run(features)
    print('- NP-Array created')
        
    # generate txt
    cell_id = np.argsort(features_np[0,:])[-1:][::-1]
    cell_prop = features_np[1:,cell_id]
    with open("results/Segmented-"+imagename+".txt",'w') as datei:
        datei.write(str(cell_id)+'\n')
        datei.write(str(cell_prop[:,0])+'\n')
    print('- txt generated')
    
# Function for Heatmap
def bildanalyse_heatmap(imagename,nmax,endung):
    
    # Load images
    images = np.zeros([nmax+1,299,299,3])
    # original image
    image = img.imread("images/output_" + imagename + "/"+ imagename + endung)
    image = image[:,:,0:3]
    image_res = resize(image, (299, 299))
    images[0,:,:,:] = image_res[:,:,:]
    # heatmap images
    for i in range(nmax):
        image = img.imread("images/output_"+imagename+"/"+imagename+"_output_"+str(i)+".jpg")
        image = image[:,:,0:3]
        image_res = resize(image, (299, 299))
        images[i+1,:,:,:] = image_res[:,:,:]
    print('- Images loaded')

    # Run PlaNet
    features = module(images)
    print('- Network done')

    # create Numpy-Array
    with tf.compat.v1.Session() as sess:
        features_np = sess.run(features)
    print('- NP-Array created')
        
    # generate txt
    cell_id = np.argsort(features_np[0,:])[-1:][::-1]
    cell_prop = features_np[1:,cell_id]
    with open("results/Heatmap-"+imagename+".txt",'w') as datei:
        datei.write(str(cell_id)+'\n')
        datei.write(str(cell_prop[:,0])+'\n')
    print('- txt generated')


In [3]:
# Cell 3: for Heatmaps
names = ["See_Sommer"] # names of the Images    
datatyp = [".jpg"] # imagetyp
n = [96] # Number of Heatmap Images (maximum number in the names + 1)

for i in range(0,len(names)):
    print(('Picture '+ names[i]+' is processed: '+str(i+1)+' from '+str(len(names))))
    bildanalyse_heatmap(names[i],n[i],datatyp[i])

Picture See_Sommer is processed: 1 from 1


- Images loaded
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


- Network done
- NP-Array created
- txt generated


In [4]:
# Cell 4: for Superpixel
names = ["Sommer"] # names of the Images    
datatyp = [".png"] # imagetyp
n = [15] # Number of superpixel Images (maximum number in the names)

for i in range(0,len(names)):
    print(('Picture '+ names[i]+' is processed: '+str(i+1)+' from '+str(len(names))))
    bildanalyse_superpixel(names[i],n[i],datatyp[i])

Picture Sommer is processed: 1 from 1
- Images loaded
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


- Network done
- NP-Array created
- txt generated


In [5]:
# Cell 5: for Geoplots
images=["See_Sommer.jpg"] # names of the images
bildanalyse_geoplots(images)

Images loaded
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Network done
NP-Array created
txt generated
